- zero shot
- gemini 1.5 flash


In [3]:
import pandas as pd
import os
import requests
import json
import time
from tqdm.notebook import tqdm
import re
from dotenv import load_dotenv

# --- 1. 設定とAPIキーの読み込み ---

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if API_KEY:
    print("✅ APIキーを.envファイルから正常に読み込みました。")
else:
    print("❌ .envファイルからGEMINI_API_KEYが見つかりませんでした。")
    exit()

MODEL_NAME = "gemini-1.5-flash"
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={API_KEY}"

# 入力ファイルとプロンプトファイル
INPUT_CSV = '../data/processed/samples_with_text.csv'
PROMPT_FILE_PATH = '../prompts/zero_shot_fulltext.txt'
# 既存のLLM予測結果ファイルに結果を追記する
OUTPUT_FILE = '../data/processed/prediction_llm.csv'

# --- 2. プロンプトとデータの準備 ---
try:
    with open(PROMPT_FILE_PATH, 'r', encoding='utf-8') as f:
        PROMPT_TEMPLATE = f.read()
    print(f"✅ プロンプトファイル '{os.path.basename(PROMPT_FILE_PATH)}' を正常に読み込みました。")
    
    df_to_process = pd.read_csv(INPUT_CSV)
    df_to_process.dropna(subset=['full_text'], inplace=True)
    df_to_process.drop_duplicates(subset=['citing_paper_doi'], inplace=True, keep='first')
    df_to_process.reset_index(drop=True, inplace=True)
    
    print(f"合計 {len(df_to_process)} 件の論文に対してLLM(Full-Text)の判定を実行します。")
except Exception as e:
    print(f"❌ エラー: データの読み込みに失敗しました。 {e}")
    df_to_process = pd.DataFrame()

# --- 3. LLMによる判定の本番実行 ---
predictions = []
if PROMPT_TEMPLATE and not df_to_process.empty:
    for index, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc="LLM(Full-Text)で判定中"):
        
        full_text = row.get('full_text', 'フルテキストが見つかりませんでした。')
        
        prompt = PROMPT_TEMPLATE.format(
            cited_data_paper_title=row['cited_data_paper_title'],
            citing_paper_title=row['citing_paper_title'],
            full_text=full_text[:30000] # トークン数上限を考慮
        )
        
        payload = {"contents": [{"parts": [{"text": prompt}]}]}
        
        try:
            response = requests.post(API_URL, json=payload, timeout=120)
            response.raise_for_status()
            
            response_json = response.json()
            response_text = response_json['candidates'][0]['content']['parts'][0]['text']
            
            match = re.search(r"\{.*\}", response_text, re.DOTALL)
            json_text = match.group(0) if match else response_text
            final_json = json.loads(json_text)
            
            decision = final_json.get('decision')
            prediction = 1 if decision == "Used" else 0
            
        except Exception as e:
            print(f"\nDOI {row['citing_paper_doi']} の処理でエラー: {e}")
            prediction = -1
            
        predictions.append(prediction)
        time.sleep(1)

# --- 4. 結果の保存 ---
if len(predictions) == len(df_to_process):
    # 今回の予測結果をDataFrameにまとめる
    df_fulltext_predictions = pd.DataFrame({
        'citing_paper_doi': df_to_process['citing_paper_doi'],
        'prediction_rule3_fulltext': predictions
    })
    
    # 既存のLLM予測結果ファイル(アブストラクト版)を読み込む
    try:
        output_df = pd.read_csv(OUTPUT_FILE)
    except FileNotFoundError:
        print(f"警告: '{OUTPUT_FILE}' が見つかりません。キー情報のみで新しいファイルを作成します。")
        output_df = df_to_process[['citing_paper_eid', 'citing_paper_doi', 'citing_paper_title', 'cited_data_paper_title']].copy()

    # 既存のデータと新しい予測結果をマージ（結合）
    # 既に同名カラムがあれば上書きし、なければ追加する
    if 'prediction_rule3_fulltext' in output_df.columns:
        output_df.drop(columns=['prediction_rule3_fulltext'], inplace=True)
        
    output_df = pd.merge(output_df, df_fulltext_predictions, on='citing_paper_doi', how='left')
    
    # ファイルに上書き保存
    output_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"\n処理完了。LLM(Full-Text)の予測結果を '{OUTPUT_FILE}' に追加・更新しました。")
    print("\n--- 保存された結果の内訳（フルテキスト版） ---")
    print(output_df['prediction_rule3_fulltext'].value_counts())
    print("\n--- 保存されたデータの先頭5件 ---")
    print(output_df.head())
else:
    print("処理が正常に完了しませんでした。")

✅ APIキーを.envファイルから正常に読み込みました。
✅ プロンプトファイル 'zero_shot_fulltext.txt' を正常に読み込みました。
合計 200 件の論文に対してLLM(Full-Text)の判定を実行します。


LLM(Full-Text)で判定中:   0%|          | 0/200 [00:00<?, ?it/s]


処理完了。LLM(Full-Text)の予測結果を '../data/processed/prediction_llm.csv' に追加・更新しました。

--- 保存された結果の内訳（フルテキスト版） ---
prediction_rule3_fulltext
0    141
1     53
Name: count, dtype: int64

--- 保存されたデータの先頭5件 ---
     citing_paper_eid                 citing_paper_doi  \
0  2-s2.0-85211640167  10.1016/j.marpolbul.2024.117442   
1  2-s2.0-85210281314   10.1016/j.jaridenv.2024.105282   
2  2-s2.0-85171680307   10.1016/j.revpalbo.2023.104989   
3  2-s2.0-85142708771     10.1016/j.foreco.2022.120653   
4  2-s2.0-85194770743    10.1016/j.jnucmat.2024.155194   

                                  citing_paper_title  \
0  Multi-indicator assessment of heavy metal poll...   
1  Potential effects of climate change on cacti d...   
2  Approaches to pollen taxonomic harmonisation i...   
3  Allometric equations to estimate the dry mass ...   
4  Microstructural evolution in doped high entrop...   

                              cited_data_paper_title  \
0  Pollution load index for heavy metals in Mian-...   
1 

- few shot
- cot
- gemini 1.5 flash


In [14]:
import pandas as pd
import os
import requests
import json
import time
from tqdm.notebook import tqdm
import re
from dotenv import load_dotenv

# --- 1. 設定とAPIキーの読み込み ---

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if API_KEY:
    print("✅ APIキーを.envファイルから正常に読み込みました。")
else:
    print("❌ .envファイルからGEMINI_API_KEYが見つかりませんでした。")
    exit()

MODEL_NAME = "gemini-1.5-flash"
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={API_KEY}"

# 入力ファイルとプロンプトファイル
INPUT_CSV = '../data/processed/samples_with_text.csv'
# 【変更点】Few-shot & CoTプロンプトを読み込む
PROMPT_FILE_PATH = '../prompts/few_shot_cot_fulltext.txt' 
# 既存のLLM予測結果ファイルに結果を追記する
OUTPUT_FILE = '../data/processed/prediction_llm.csv'

# --- 2. プロンプトとデータの準備 ---
try:
    with open(PROMPT_FILE_PATH, 'r', encoding='utf-8') as f:
        PROMPT_TEMPLATE = f.read()
    print(f"✅ プロンプトファイル '{os.path.basename(PROMPT_FILE_PATH)}' を正常に読み込みました。")
    
    df_to_process = pd.read_csv(INPUT_CSV)
    df_to_process.dropna(subset=['full_text'], inplace=True)
    df_to_process.drop_duplicates(subset=['citing_paper_doi'], inplace=True, keep='first')
    df_to_process.reset_index(drop=True, inplace=True)
    
    print(f"合計 {len(df_to_process)} 件の論文に対してLLM(Full-Text, Few-shot)の判定を実行します。")
except Exception as e:
    print(f"❌ エラー: データの読み込みに失敗しました。 {e}")
    df_to_process = pd.DataFrame()

# --- 3. LLMによる判定の本番実行 ---
predictions = []
if PROMPT_TEMPLATE and not df_to_process.empty:
    for index, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc="LLM(Full-Text, Few-shot)で判定中"):
        
        full_text = row.get('full_text', 'フルテキストが見つかりませんでした。')
        
        prompt = PROMPT_TEMPLATE.format(
            cited_data_paper_title=row['cited_data_paper_title'],
            citing_paper_title=row['citing_paper_title'],
            full_text=full_text[:30000] # トークン数上限を考慮
        )
        
        payload = {"contents": [{"parts": [{"text": prompt}]}]}
        
        try:
            response = requests.post(API_URL, json=payload, timeout=120)
            response.raise_for_status()
            
            response_json = response.json()
            response_text = response_json['candidates'][0]['content']['parts'][0]['text']
            
            match = re.search(r"\{.*\}", response_text, re.DOTALL)
            json_text = match.group(0) if match else response_text
            final_json = json.loads(json_text)
            
            decision = final_json.get('decision')
            prediction = 1 if decision == "Used" else 0
            
        except Exception as e:
            print(f"\nDOI {row['citing_paper_doi']} の処理でエラー: {e}")
            prediction = -1
            
        predictions.append(prediction)
        time.sleep(1)

# --- 4. 結果の保存 ---
if len(predictions) == len(df_to_process):
    # 今回の予測結果をDataFrameにまとめる
    df_new_predictions = pd.DataFrame({
        'citing_paper_doi': df_to_process['citing_paper_doi'],
        # 【変更点】新しいカラム名で結果を保存
        'prediction_rule3_fulltext_few_shot': predictions 
    })
    
    # 既存のLLM予測結果ファイルを読み込む
    try:
        output_df = pd.read_csv(OUTPUT_FILE)
    except FileNotFoundError:
        print(f"警告: '{OUTPUT_FILE}' が見つかりません。キー情報のみで新しいファイルを作成します。")
        output_df = df_to_process[['citing_paper_eid', 'citing_paper_doi', 'citing_paper_title', 'cited_data_paper_title']].copy()

    # 既存のデータと新しい予測結果をマージ（結合）
    if 'prediction_rule3_fulltext_few_shot' in output_df.columns:
        output_df.drop(columns=['prediction_rule3_fulltext_few_shot'], inplace=True)
        
    output_df = pd.merge(output_df, df_new_predictions, on='citing_paper_doi', how='left')
    
    # ファイルに上書き保存
    output_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"\n処理完了。LLM(Full-Text, Few-shot)の予測結果を '{OUTPUT_FILE}' に追加・更新しました。")
    print("\n--- 保存された結果の内訳（Few-shot版） ---")
    print(output_df['prediction_rule3_fulltext_few_shot'].value_counts())
else:
    print("処理が正常に完了しませんでした。")

✅ APIキーを.envファイルから正常に読み込みました。
✅ プロンプトファイル 'few_shot_cot_fulltext.txt' を正常に読み込みました。
合計 200 件の論文に対してLLM(Full-Text, Few-shot)の判定を実行します。


LLM(Full-Text, Few-shot)で判定中:   0%|          | 0/200 [00:00<?, ?it/s]


処理完了。LLM(Full-Text, Few-shot)の予測結果を '../data/processed/prediction_llm.csv' に追加・更新しました。

--- 保存された結果の内訳（Few-shot版） ---
prediction_rule3_fulltext_few_shot
0    123
1     77
Name: count, dtype: int64


test


In [15]:
import os
import requests
import json
import re
from dotenv import load_dotenv

# --- 1. 設定とAPIキーの読み込み ---

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    print("❌ .envファイルからGEMINI_API_KEYが見つかりませんでした。")
    exit()

MODEL_NAME = "gemini-2.5-flash"
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={API_KEY}"
PROMPT_FILE_PATH = '../prompts/few_shot_cot_fulltext.txt' # ご自身のプロンプトファイルへのパス

# --- 2. プロンプトテンプレートの読み込み ---
try:
    with open(PROMPT_FILE_PATH, 'r', encoding='utf-8') as f:
        PROMPT_TEMPLATE = f.read()
    print(f"✅ プロンプトファイル '{os.path.basename(PROMPT_FILE_PATH)}' を読み込みました。")
except Exception as e:
    print(f"❌ プロンプトファイルの読み込みに失敗しました: {e}")
    exit()

# --- 3. 確認したいサンプルデータ ---
# ここの内容を書き換えることで、色々なケースをテストできます。
sample_row = {
    'cited_data_paper_title': "ImageNet Large Scale Visual Recognition Challenge",
    'citing_paper_title': "Deep Residual Learning for Image Recognition",
    'full_text': (
        "4. Experiments. We conduct comprehensive experiments on the ImageNet 2012 classification dataset "
        "that consists of 1.28 million training images and 50k validation images. "
        "We evaluate both training error and validation error. "
        "Our ResNet-152 model has a top-1 validation error of 19.38%. This result won the 1st place in the ILSVRC 2015."
    )
}

# --- 4. 単一リクエストのテスト実行 ---

print("\n" + "="*50)
print(" singolo APIリクエストのテストを開始します")
print("="*50)

# 4.1. プロンプトの生成
# トークン数上限を考慮し、実際のスクリプトに合わせてテキストを切り詰めます
final_prompt = PROMPT_TEMPLATE.format(
    cited_data_paper_title=sample_row['cited_data_paper_title'],
    citing_paper_title=sample_row['citing_paper_title'],
    full_text=sample_row['full_text'][:30000] 
)

print("\n✅ 1. LLMへの送信プロンプト（最終版）")
print("-" * 40)
print(final_prompt)
print("-" * 40)


# 4.2. APIリクエストの実行
payload = {"contents": [{"parts": [{"text": final_prompt}]}]}
print("\n🔍 2. APIにリクエストを送信中...")

try:
    response = requests.post(API_URL, json=payload, timeout=120)
    response.raise_for_status()  # エラーがあればここで例外を発生させる
    
    print("\n✅ 3. LLMからのRAWレスポンス (JSON全体)")
    print("-" * 40)
    # レスポンスのJSONを整形して表示
    raw_response_json = response.json()
    print(json.dumps(raw_response_json, indent=2, ensure_ascii=False))
    print("-" * 40)

    # 4.3. レスポンスの解析
    response_text = raw_response_json['candidates'][0]['content']['parts'][0]['text']
    
    # JSON部分を抽出
    match = re.search(r"\{.*\}", response_text, re.DOTALL)
    json_text = match.group(0) if match else response_text
    final_json = json.loads(json_text)
    
    print("\n✅ 4. パース（解析）後の主要な結果")
    print("-" * 40)
    print(json.dumps(final_json, indent=2, ensure_ascii=False))
    print("-" * 40)
    
    # 4.4. 最終判定
    decision = final_json.get('decision', 'N/A')
    prediction = 1 if decision == "Used" else (0 if decision == "Not Used" else -1)
    
    print(f"\n🎉 5. 最終判定")
    print("-" * 40)
    print(f"Decision: '{decision}'  =>  Prediction: {prediction}")
    print("-" * 40)

except requests.exceptions.RequestException as e:
    print(f"\n❌ APIリクエストでエラーが発生しました: {e}")
except Exception as e:
    print(f"\n❌ 処理中に予期せぬエラーが発生しました: {e}")

✅ プロンプトファイル 'few_shot_cot_fulltext.txt' を読み込みました。

 singolo APIリクエストのテストを開始します

✅ 1. LLMへの送信プロンプト（最終版）
----------------------------------------
You are an expert AI assistant specializing in the analysis of academic papers.

### Instructions
Following the 【Thought Process】 and 【Criteria】 below, determine whether the "Citing Paper" provided in the 【Input Data】 actually uses the data from the specified "Cited Data Paper".
Refer to the 【Examples】 and ensure the final output is in JSON format, containing only the "Used" or "Not Used" decision.

### 【Thought Process】
First, read the full text of the Citing Paper, paying close attention to sections like "Methods", "Experiment", and "Results" to understand the research objective and methodology.

Next, search for any mentions related to the "Cited Data Paper Title" within that context.

Carefully determine if the mention is used directly to support the paper's conclusions (e.g., for performance evaluation, analysis, or comparison) or if it is

- few shot
- cot
- gemini 2.5 flash


In [16]:
import pandas as pd
import os
import requests
import json
import time
from tqdm.notebook import tqdm
import re
from dotenv import load_dotenv

# --- 1. 設定とAPIキーの読み込み ---

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if API_KEY:
    print("✅ APIキーを.envファイルから正常に読み込みました。")
else:
    print("❌ .envファイルからGEMINI_API_KEYが見つかりませんでした。")
    exit()

# 【変更点】将来のモデル名に変更
MODEL_NAME = "gemini-2.5-flash" 
# 注意: このモデルは現時点では存在しないため、実行するとエラーになります。
# 将来的にモデルがリリースされた際に、この名前が正しいかを確認してください。

API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={API_KEY}"
print(f"✅ 使用モデル: {MODEL_NAME}")

# 入力ファイルとプロンプトファイル
INPUT_CSV = '../data/processed/samples_with_text.csv'
PROMPT_FILE_PATH = '../prompts/few_shot_cot_fulltext.txt'
OUTPUT_FILE = '../data/processed/prediction_llm.csv'

# --- 2. プロンプトとデータの準備 ---
try:
    with open(PROMPT_FILE_PATH, 'r', encoding='utf-8') as f:
        PROMPT_TEMPLATE = f.read()
    print(f"✅ プロンプトファイル '{os.path.basename(PROMPT_FILE_PATH)}' を正常に読み込みました。")
    
    df_to_process = pd.read_csv(INPUT_CSV)
    df_to_process.dropna(subset=['full_text'], inplace=True)
    df_to_process.drop_duplicates(subset=['citing_paper_doi'], inplace=True, keep='first')
    df_to_process.reset_index(drop=True, inplace=True)
    
    print(f"合計 {len(df_to_process)} 件の論文に対してLLM({MODEL_NAME})の判定を実行します。")
except Exception as e:
    print(f"❌ エラー: データの読み込みに失敗しました。 {e}")
    df_to_process = pd.DataFrame()

# --- 3. LLMによる判定の本番実行 ---
predictions = []
if PROMPT_TEMPLATE and not df_to_process.empty:
    for index, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc=f"LLM({MODEL_NAME})で判定中"):
        
        full_text = row.get('full_text', 'フルテキストが見つかりませんでした。')
        
        prompt = PROMPT_TEMPLATE.format(
            cited_data_paper_title=row['cited_data_paper_title'],
            citing_paper_title=row['citing_paper_title'],
            full_text=full_text[:30000] # トークン数上限を考慮
        )
        
        payload = {"contents": [{"parts": [{"text": prompt}]}]}
        
        try:
            response = requests.post(API_URL, json=payload, timeout=180)
            response.raise_for_status()
            
            response_json = response.json()
            response_text = response_json['candidates'][0]['content']['parts'][0]['text']
            
            match = re.search(r"\{.*\}", response_text, re.DOTALL)
            json_text = match.group(0) if match else response_text
            final_json = json.loads(json_text)
            
            decision = final_json.get('decision')
            prediction = 1 if decision == "Used" else 0
            
        except Exception as e:
            print(f"\nDOI {row['citing_paper_doi']} の処理でエラー: {e}")
            prediction = -1
            
        predictions.append(prediction)
        time.sleep(1.5)

# --- 4. 結果の保存 ---
if len(predictions) == len(df_to_process):
    # 今回の予測結果をDataFrameにまとめる
    new_column_name = f"prediction_rule3_{MODEL_NAME.replace('.', '_')}" # モデル名からカラム名を生成
    df_new_predictions = pd.DataFrame({
        'citing_paper_doi': df_to_process['citing_paper_doi'],
        new_column_name: predictions
    })
    
    try:
        output_df = pd.read_csv(OUTPUT_FILE)
    except FileNotFoundError:
        output_df = df_to_process[['citing_paper_eid', 'citing_paper_doi', 'citing_paper_title', 'cited_data_paper_title']].copy()

    if new_column_name in output_df.columns:
        output_df.drop(columns=[new_column_name], inplace=True)
        
    output_df = pd.merge(output_df, df_new_predictions, on='citing_paper_doi', how='left')
    
    output_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"\n処理完了。LLM({MODEL_NAME})の予測結果を '{OUTPUT_FILE}' に追加・更新しました。")
    print(f"\n--- 保存された結果の内訳（{MODEL_NAME}版） ---")
    print(output_df[new_column_name].value_counts())
else:
    print("処理が正常に完了しませんでした。")

✅ APIキーを.envファイルから正常に読み込みました。
✅ 使用モデル: gemini-2.5-flash
✅ プロンプトファイル 'few_shot_cot_fulltext.txt' を正常に読み込みました。
合計 200 件の論文に対してLLM(gemini-2.5-flash)の判定を実行します。


LLM(gemini-2.5-flash)で判定中:   0%|          | 0/200 [00:00<?, ?it/s]


DOI 10.1016/j.oneear.2025.101197 の処理でエラー: HTTPSConnectionPool(host='generativelanguage.googleapis.com', port=443): Read timed out. (read timeout=180)

処理完了。LLM(gemini-2.5-flash)の予測結果を '../data/processed/prediction_llm.csv' に追加・更新しました。

--- 保存された結果の内訳（gemini-2.5-flash版） ---
prediction_rule3_gemini-2_5-flash
 0    124
 1     75
-1      1
Name: count, dtype: int64


リトライ用


In [18]:
import pandas as pd
import os
import requests
import json
import time
from tqdm.notebook import tqdm
import re
from dotenv import load_dotenv

# --- 1. 設定とAPIキーの読み込み ---

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if API_KEY:
    print("✅ APIキーを.envファイルから正常に読み込みました。")
else:
    print("❌ .envファイルからGEMINI_API_KEYが見つかりませんでした。")
    exit()

# --- 【重要】再試行するモデルとプロンプトを設定 ---
# モデル名を 'gemini-1.5-flash' または 'gemini-1.5-pro' など、再試行したいモデル名に変更してください
MODEL_NAME = "gemini-2.5-flash" 
# 再試行に使用するプロンプトファイルを指定してください
PROMPT_FILE_PATH = '../prompts/few_shot_cot_fulltext.txt' 
# 再試行する対象のカラム名を指定してください
COLUMN_TO_RETRY = 'prediction_rule3_gemini-2_5-flash'

API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={API_KEY}"
print(f"✅ 使用モデル: {MODEL_NAME}")

# 入力ファイル
SAMPLES_CSV = '../data/processed/samples_with_text.csv'
PREDICTIONS_CSV = '../data/processed/prediction_llm.csv'

# --- 2. プロンプトとデータの準備 ---
try:
    with open(PROMPT_FILE_PATH, 'r', encoding='utf-8') as f:
        PROMPT_TEMPLATE = f.read()
    
    df_samples = pd.read_csv(SAMPLES_CSV)
    df_predictions = pd.read_csv(PREDICTIONS_CSV)

    # --- 再試行対象の論文を特定 ---
    if COLUMN_TO_RETRY not in df_predictions.columns:
        print(f"❌ エラー: '{COLUMN_TO_RETRY}' カラムが予測結果ファイルに見つかりません。")
        df_to_retry = pd.DataFrame()
    else:
        # 予測結果が-1の行のDOIを取得
        retry_dois = df_predictions[df_predictions[COLUMN_TO_RETRY] == -1]['citing_paper_doi']
        # samples_with_text.csvから、そのDOIに一致する行を抽出
        df_to_retry = df_samples[df_samples['citing_paper_doi'].isin(retry_dois)].copy()
    
    if not df_to_retry.empty:
        print(f"✅ エラー(-1)が記録された {len(df_to_retry)} 件を特定しました。再試行を開始します。")
    else:
        print("✅ エラー(-1)が記録された論文はありませんでした。処理は不要です。")

except Exception as e:
    print(f"❌ エラー: データの読み込みに失敗しました。 {e}")
    df_to_retry = pd.DataFrame()

# --- 3. 失敗したタスクに対してのみLLM判定を再実行 ---
retry_results = []
if not df_to_retry.empty and PROMPT_TEMPLATE:
    for index, row in tqdm(df_to_retry.iterrows(), total=len(df_to_retry), desc=f"{MODEL_NAME}で再試行中"):
        
        full_text = row.get('full_text', 'フルテキストが見つかりませんでした。')
        
        prompt = PROMPT_TEMPLATE.format(
            cited_data_paper_title=row['cited_data_paper_title'],
            citing_paper_title=row['citing_paper_title'],
            full_text=full_text[:30000]
        )
        
        payload = {"contents": [{"parts": [{"text": prompt}]}]}
        
        try:
            response = requests.post(API_URL, json=payload, timeout=180)
            response.raise_for_status()
            
            response_json = response.json()
            response_text = response_json['candidates'][0]['content']['parts'][0]['text']
            
            match = re.search(r"\{.*\}", response_text, re.DOTALL)
            json_text = match.group(0) if match else response_text
            final_json = json.loads(json_text)
            
            prediction = 1 if final_json.get('decision') == "Used" else 0
            
        except Exception as e:
            print(f"\nDOI {row['citing_paper_doi']} の再試行中にエラー: {e}")
            prediction = -1 # 再試行しても失敗した場合は-1のまま
            
        retry_results.append({
            'citing_paper_doi': row['citing_paper_doi'],
            COLUMN_TO_RETRY: prediction
        })
        time.sleep(1.5)

# --- 4. 結果の更新と保存 ---
if retry_results:
    df_retry_results = pd.DataFrame(retry_results)
    
    # 元の予測結果のDOIをインデックスに設定
    df_predictions.set_index('citing_paper_doi', inplace=True)
    # 再試行結果のDOIをインデックスに設定
    df_retry_results.set_index('citing_paper_doi', inplace=True)
    
    # 再試行の結果で元のデータを更新
    df_predictions.update(df_retry_results)
    
    # インデックスをリセット
    df_predictions.reset_index(inplace=True)
    
    df_predictions.to_csv(PREDICTIONS_CSV, index=False, encoding='utf-8-sig')

    print(f"\n再試行完了。'{PREDICTIONS_CSV}' を更新しました。")
    print(f"\n--- 最新の予測結果の内訳（{MODEL_NAME}版） ---")
    print(df_predictions[COLUMN_TO_RETRY].value_counts())
else:
    print("再試行するデータがありませんでした。")

✅ APIキーを.envファイルから正常に読み込みました。
✅ 使用モデル: gemini-2.5-flash
✅ エラー(-1)が記録された 1 件を特定しました。再試行を開始します。


gemini-2.5-flashで再試行中:   0%|          | 0/1 [00:00<?, ?it/s]


再試行完了。'../data/processed/prediction_llm.csv' を更新しました。

--- 最新の予測結果の内訳（gemini-2.5-flash版） ---
prediction_rule3_gemini-2_5-flash
0    125
1     75
Name: count, dtype: int64


- zero shot
- gemini 2.5 flash


In [19]:
import pandas as pd
import os
import requests
import json
import time
from tqdm.notebook import tqdm
import re
from dotenv import load_dotenv

# --- 1. 設定とAPIキーの読み込み ---

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

if API_KEY:
    print("✅ APIキーを.envファイルから正常に読み込みました。")
else:
    print("❌ .envファイルからGEMINI_API_KEYが見つかりませんでした。")
    exit()

# 【重要】実行するモデル名と、それに対応するプロンプトファイルを指定
MODEL_NAME = "gemini-2.5-flash" 
PROMPT_FILE_PATH = '../prompts/zero_shot_fulltext.txt' 

# APIエンドポイント
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent?key={API_KEY}"
print(f"✅ 使用モデル: {MODEL_NAME}")

# 入力ファイルと出力ファイル
INPUT_CSV = '../data/processed/samples_with_text.csv'
OUTPUT_FILE = '../data/processed/prediction_llm.csv'

# --- 2. プロンプトとデータの準備 ---
try:
    with open(PROMPT_FILE_PATH, 'r', encoding='utf-8') as f:
        PROMPT_TEMPLATE = f.read()
    print(f"✅ プロンプトファイル '{os.path.basename(PROMPT_FILE_PATH)}' を正常に読み込みました。")
    
    df_to_process = pd.read_csv(INPUT_CSV)
    df_to_process.dropna(subset=['full_text'], inplace=True)
    df_to_process.drop_duplicates(subset=['citing_paper_doi'], inplace=True, keep='first')
    df_to_process.reset_index(drop=True, inplace=True)
    
    print(f"合計 {len(df_to_process)} 件の論文に対してLLM({MODEL_NAME}, Zero-shot)の判定を実行します。")
except Exception as e:
    print(f"❌ エラー: データの読み込みに失敗しました。 {e}")
    df_to_process = pd.DataFrame()

# --- 3. LLMによる判定の本番実行 ---
predictions = []
if PROMPT_TEMPLATE and not df_to_process.empty:
    for index, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc=f"LLM({MODEL_NAME}, Zero-shot)で判定中"):
        
        full_text = row.get('full_text', 'フルテキストが見つかりませんでした。')
        
        prompt = PROMPT_TEMPLATE.format(
            cited_data_paper_title=row['cited_data_paper_title'],
            citing_paper_title=row['citing_paper_title'],
            full_text=full_text[:30000] # トークン数上限を考慮
        )
        
        payload = {"contents": [{"parts": [{"text": prompt}]}]}
        
        try:
            response = requests.post(API_URL, json=payload, timeout=180)
            response.raise_for_status()
            
            response_json = response.json()
            response_text = response_json['candidates'][0]['content']['parts'][0]['text']
            
            match = re.search(r"\{.*\}", response_text, re.DOTALL)
            json_text = match.group(0) if match else response_text
            final_json = json.loads(json_text)
            
            decision = final_json.get('decision')
            prediction = 1 if decision == "Used" else 0
            
        except Exception as e:
            print(f"\nDOI {row['citing_paper_doi']} の処理でエラー: {e}")
            prediction = -1
            
        predictions.append(prediction)
        time.sleep(1.5) # APIへの負荷を考慮

# --- 4. 結果の保存 ---
if len(predictions) == len(df_to_process):
    # 今回の予測結果をDataFrameにまとめる
    new_column_name = f"prediction_rule3_{MODEL_NAME.replace('.', '_')}_zeroshot"
    df_new_predictions = pd.DataFrame({
        'citing_paper_doi': df_to_process['citing_paper_doi'],
        new_column_name: predictions
    })
    
    # 既存のLLM予測結果ファイルを読み込む
    try:
        output_df = pd.read_csv(OUTPUT_FILE)
    except FileNotFoundError:
        output_df = df_to_process[['citing_paper_eid', 'citing_paper_doi', 'citing_paper_title', 'cited_data_paper_title']].copy()

    # 既存のデータと新しい予測結果をマージ（結合）
    if new_column_name in output_df.columns:
        output_df.drop(columns=[new_column_name], inplace=True)
        
    output_df = pd.merge(output_df, df_new_predictions, on='citing_paper_doi', how='left')
    
    # ファイルに上書き保存
    output_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print(f"\n処理完了。LLM({MODEL_NAME}, Zero-shot)の予測結果を '{OUTPUT_FILE}' に追加・更新しました。")
    print(f"\n--- 保存された結果の内訳（{MODEL_NAME}, Zero-shot版） ---")
    print(output_df[new_column_name].value_counts())
else:
    print("処理が正常に完了しませんでした。")

✅ APIキーを.envファイルから正常に読み込みました。
✅ 使用モデル: gemini-2.5-flash
✅ プロンプトファイル 'zero_shot_fulltext.txt' を正常に読み込みました。
合計 200 件の論文に対してLLM(gemini-2.5-flash, Zero-shot)の判定を実行します。


LLM(gemini-2.5-flash, Zero-shot)で判定中:   0%|          | 0/200 [00:00<?, ?it/s]


処理完了。LLM(gemini-2.5-flash, Zero-shot)の予測結果を '../data/processed/prediction_llm.csv' に追加・更新しました。

--- 保存された結果の内訳（gemini-2.5-flash, Zero-shot版） ---
prediction_rule3_gemini-2_5-flash_zeroshot
0    107
1     93
Name: count, dtype: int64
